In [1]:
import keras.backend as K
from keras.engine.topology import Layer, InputSpec
from keras.layers import Input, Dense, Flatten, Dropout
from keras.models import Model, Sequential
from keras.datasets import mnist
from keras.optimizers import SGD
import numpy as np
from sklearn.cluster import KMeans
from scipy.optimize import linear_sum_assignment
from keras.initializers import RandomNormal
from keras.callbacks import LearningRateScheduler

Using TensorFlow backend.


In [2]:
class ClusteringLayer(Layer):
    
    
    # should have output dim
    # weights are centroids
    def __init__(self, output_dim, input_dim=None, weights=None, **kwargs):
        self.output_dim = output_dim
        self.input_dim = input_dim
        
        # kmeans cluster centre locations
        self.initial_weights = weights
        self.input_spec = [InputSpec(ndim=2)]

        if self.input_dim:
            kwargs['input_shape'] = (self.input_dim,)
        super(ClusteringLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 2
        input_dim = input_shape[1]
        self.input_spec = [InputSpec(dtype=K.floatx(),
                                     shape=(None, input_dim))]

        self.W = K.variable(self.initial_weights)
        self.trainable_weights = [self.W]

    def call(self, x, mask=None):
        q = 1.0/(1.0 + K.sqrt(K.sum(K.square(K.expand_dims(x, 1) - self.W), axis=2))**2)
        q = K.transpose(K.transpose(q)/K.sum(q, axis=1))
        return q

    def get_output_shape_for(self, input_shape):
        assert input_shape and len(input_shape) == 2
        return (input_shape[0], self.output_dim)

    def compute_output_shape(self, input_shape):
        assert input_shape and len(input_shape) == 2
        return (input_shape[0], self.output_dim)

    def get_config(self):
        config = {'output_dim': self.output_dim,
                  'input_dim': self.input_dim}
        base_config = super(ClusteringLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))
        

In [3]:
class DeepEmbeddingClustering(object):
    
    # initialize class
    def __init__(self, batch_size=256, **kwargs):
        super(DeepEmbeddingClustering, self).__init__()
        
        # MNIST data is 28x28 -> if flatten would be 784
        self.input_dim = 784
        # number of clusters as stated in the paper
        self.n_clusters = 10
        self.batch_size = batch_size
        
        self.learning_rate = 0.1
        
        # how the dimensions will be changed during encoding
        self.dims = [self.input_dim, 500, 500, 2000, 10]
        
        # input layer
        self.input_layer = Input(shape=(self.input_dim,))
        dropout_fraction = 0.2
        std_deviation = 0.01
        
        self.layer_wise = []
        self.encoders = []
        self.decoders = []
        
        # creating layer-wise autoencoder
        
        for i in range(1, len(self.dims)):
            
            e_activation = 'linear' if i == (len(self.dims) - 1) else 'relu'
            
            encoder = Dense(self.dims[i], activation=e_activation,
                           input_shape=(self.dims[i-1],),
                           kernel_initializer=RandomNormal(mean=0.0, stddev=std_deviation, seed=None),
                            bias_initializer='zeros')
            self.encoders.append(encoder)
            
            d_activation = 'linear' if i == 1 else 'relu'
            
            decoder = Dense(self.dims[i-1], activation=d_activation,
                           input_shape=(self.dims[i],),
                           kernel_initializer=RandomNormal(mean=0.0, stddev=std_deviation, seed=None),
                            bias_initializer='zeros')
            
            self.decoders.append(decoder)
            
            autoencoder = Sequential([
                Dropout(dropout_fraction),
                encoder,
                Dropout(dropout_fraction),
                decoder
            ])
            autoencoder.compile(loss='mse',
                               optimizer=SGD(lr=self.learning_rate, decay=0, momentum=0.9))
            self.layer_wise.append(autoencoder)
        
        # autoencoder for fine-tuning
        self.encoder = Sequential(self.encoders)
        self.encoder.compile(loss='mse',
                            optimizer=SGD(lr=self.learning_rate, decay=0, momentum=0.9))
        # decoders are used in reverse order
        self.decoders.reverse()
        self.autoencoder = Sequential(self.encoders + self.decoders)
        self.autoencoder.compile(loss='mse', optimizer=SGD(lr=self.learning_rate, decay=0, momentum=0.9))
        
    
    def initialize(self, X):
        current_input = X
        
        layerwise_epochs = 50
        finetune_epochs = 50
        
        # layer-wise pretrain
        i = 0
        for autoencoder in self.layer_wise:
            if i > 0:
                weights = self.encoders[i-1].get_weights()
                dense = Dense(self.dims[i], input_shape=(current_input.shape[1],),
                             activation='relu', weights=weights)
                
                e_model = Sequential([dense])
                e_model.compile(loss='mse',
                               optimizer=SGD(lr=self.learning_rate, decay=0, momentum=0.9))
                current_input = e_model.predict(current_input)
                
            autoencoder.fit(current_input, current_input, epochs=layerwise_epochs,
                            batch_size=self.batch_size)
            self.autoencoder.layers[i].set_weights(autoencoder.layers[1].get_weights())
            self.autoencoder.layers[len(self.autoencoder.layers) - i - 1].set_weights(autoencoder.layers[-1].get_weights())
                
            i += 1
            
            
        # fine tuning
        self.autoencoder.fit(X, X, epochs=finetune_epochs, batch_size=self.batch_size)
        
        # finding centroids with k-means
        kmeans = KMeans(n_clusters=self.n_clusters, n_init=20)
        self.y_pred = kmeans.fit_predict(self.encoder.predict(X))
        self.centroids = kmeans.cluster_centers_
        
        self.model = Sequential([self.encoder,
                             ClusteringLayer(self.n_clusters,
                                                weights=self.centroids,
                                                name='clustering')])
        self.model.compile(loss='kullback_leibler_divergence', 
                           optimizer=SGD(lr=self.learning_rate, decay=0, momentum=0.9))
        
        return self.y_pred
        
    
    def calculate_pi(self, q):
        p = q ** 2 / q.sum(0)
        return (p.T / p.sum(1)).T
    
  
    def train_cluster(self, X, y, **kwargs):
        tol = 0.1
        max_iter = 100
            
        train = True
        iteration = 0
        
        while train:
            # stop when reaching maximum iteration
            if max_iter < iteration:
                train = False
            self.q = self.model.predict(X)
            self.p = self.calculate_pi(self.q)
            
            y_pred = self.q.argmax(1)
            changed_assignment = 0
            for i in range(len(y_pred)):
                if y_pred[i] != self.y_pred[i]:
                    changed_assignment += 1
            
            changed_assignment = changed_assignment / len(y_pred)
            print('Amount of dots, that changed assignment is ' + str(changed_assignment))
            
            if changed_assignment < tol:
                print('reached tolerance')
                train = False
            else:
                self.y_pred = y_pred
                
            for i in range(len(self.encoder.layers)):
                self.encoder.layers[i].set_weights(self.model.layers[0].layers[i].get_weights())
            self.centroids = self.model.layers[-1].get_weights()[0]
            
            self.model.fit(X, self.p, epochs=50, batch_size=self.batch_size)
                
            iteration += 1
        
        return self.model

In [4]:
def calculate_accuracy(y_pred, y):
    # initialize cost matrix for Hungarian algorithm
    cost_matrix = np.zeros((10, 10), dtype=np.int64)
        
    # create cost_matrix
    # counts how much each clusterization label
    # was equal
    for i in range(len(y_pred)):
        cost_matrix[y_pred[i], y[i]] += 1
            
    # algorithm minimizes cost, thus we create "inverse" matrix
    row_ind, col_ind = linear_sum_assignment(cost_matrix.max() - cost_matrix)
        
    # row_ind and col_ind corresponds to how we map
    # true labels to labels after clusterization
    # our cost_matrix contained amount of correct labelling
    accuracy = 0
    for i, j in zip(row_ind, col_ind):
        accuracy += cost_matrix[i, j]
            
    return accuracy/len(y_pred)

In [5]:
np.random.seed(1234) # set seed for deterministic ordering
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_all = np.concatenate((x_train, x_test), axis = 0)
Y = np.concatenate((y_train, y_test), axis = 0)
X = x_all.reshape(-1,x_all.shape[1]*x_all.shape[2])
    
p = np.random.permutation(X.shape[0])
X = X[p].astype(np.float32)*0.02
Y = Y[p]

In [6]:
# kmeans
kmeans = KMeans(n_clusters=10, n_init=20)
y_pred_kmeans = kmeans.fit_predict(X)
print('accuracy for k_means')
accuracy = calculate_accuracy(y_pred_kmeans, Y)
print(accuracy)

mod = DeepEmbeddingClustering(batch_size=256)

# returns labels from k-means + trained encoder
y_pred_kmeans_encoder = mod.initialize(X)

print('accuracy for k_means + encoder')
accuracy = calculate_accuracy(y_pred_kmeans_encoder, Y)
print(accuracy)

accuracy for k_means
0.5334
Epoch 1/50
70000/70000 [==============================] - 9s 124us/step - loss: 0.7270
Epoch 2/50
70000/70000 [==============================] - 8s 116us/step - loss: 0.4305
Epoch 3/50
70000/70000 [==============================] - 9s 122us/step - loss: 0.4001
Epoch 4/50
70000/70000 [==============================] - 8s 120us/step - loss: 0.3856
Epoch 5/50
70000/70000 [==============================] - 8s 121us/step - loss: 0.3764
Epoch 6/50
70000/70000 [==============================] - 8s 118us/step - loss: 0.3696
Epoch 7/50
70000/70000 [==============================] - 8s 119us/step - loss: 0.3643
Epoch 8/50
70000/70000 [==============================] - 8s 119us/step - loss: 0.3599
Epoch 9/50
70000/70000 [==============================] - 8s 118us/step - loss: 0.3563
Epoch 10/50
70000/70000 [==============================] - 9s 125us/step - loss: 0.3532
Epoch 11/50
70000/70000 [==============================] - 9s 122us/step - loss: 0.3504
Epoch 12/50
7

70000/70000 [==============================] - 6s 82us/step - loss: 0.1542
Epoch 45/50
70000/70000 [==============================] - 6s 82us/step - loss: 0.1536
Epoch 46/50
70000/70000 [==============================] - 6s 81us/step - loss: 0.1532
Epoch 47/50
70000/70000 [==============================] - 6s 81us/step - loss: 0.1527
Epoch 48/50
70000/70000 [==============================] - 6s 82us/step - loss: 0.1522
Epoch 49/50
70000/70000 [==============================] - 6s 91us/step - loss: 0.1520
Epoch 50/50
70000/70000 [==============================] - 6s 82us/step - loss: 0.1514
Epoch 1/50
70000/70000 [==============================] - 22s 316us/step - loss: 0.3369
Epoch 2/50
70000/70000 [==============================] - 23s 321us/step - loss: 0.1332
Epoch 3/50
70000/70000 [==============================] - 23s 323us/step - loss: 0.1091
Epoch 4/50
70000/70000 [==============================] - 21s 294us/step - loss: 0.0999
Epoch 5/50
70000/70000 [===========================

70000/70000 [==============================] - 4s 63us/step - loss: 0.0853
Epoch 38/50
70000/70000 [==============================] - 4s 64us/step - loss: 0.0852
Epoch 39/50
70000/70000 [==============================] - 4s 62us/step - loss: 0.0851
Epoch 40/50
70000/70000 [==============================] - 4s 63us/step - loss: 0.0851
Epoch 41/50
70000/70000 [==============================] - 5s 66us/step - loss: 0.0850
Epoch 42/50
70000/70000 [==============================] - 5s 66us/step - loss: 0.0849
Epoch 43/50
70000/70000 [==============================] - 4s 64us/step - loss: 0.0849
Epoch 44/50
70000/70000 [==============================] - 5s 74us/step - loss: 0.0848
Epoch 45/50
70000/70000 [==============================] - 4s 62us/step - loss: 0.0848
Epoch 46/50
70000/70000 [==============================] - 5s 66us/step - loss: 0.0848
Epoch 47/50
70000/70000 [==============================] - 5s 65us/step - loss: 0.0847
Epoch 48/50
70000/70000 [==============================

In [7]:
# dec model
dec_model = mod.train_cluster(X, Y)
y_pred_dec = dec_model.predict(X).argmax(1)
print('accuracy for DEC')
accuracy = calculate_accuracy(y_pred_dec, Y)
print(accuracy)

Amount of dots, that changed assignment is 0.0
reached tolerance
Epoch 1/50
70000/70000 [==============================] - 22s 311us/step - loss: 0.0297
Epoch 2/50
70000/70000 [==============================] - 20s 292us/step - loss: 0.0193
Epoch 3/50
70000/70000 [==============================] - 20s 289us/step - loss: 0.0181
Epoch 4/50
70000/70000 [==============================] - 20s 291us/step - loss: 0.0174
Epoch 5/50
70000/70000 [==============================] - 20s 289us/step - loss: 0.0169
Epoch 6/50
70000/70000 [==============================] - 20s 292us/step - loss: 0.0166
Epoch 7/50
70000/70000 [==============================] - 21s 299us/step - loss: 0.0163
Epoch 8/50
70000/70000 [==============================] - 20s 291us/step - loss: 0.0160
Epoch 9/50
70000/70000 [==============================] - 21s 294us/step - loss: 0.0158
Epoch 10/50
70000/70000 [==============================] - 21s 300us/step - loss: 0.0156
Epoch 11/50
70000/70000 [=============================